In [1]:
import sys
sys.path.insert(0, "/work/04233/sw33286/AIDA-SCRIPTS")

In [2]:
import os
import time
import random
import shutil
import dill
import numpy as np

import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell, MultiRNNCell, DropoutWrapper

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline
import colorama
from colorama import Style

from helpers import Indexer, batch
from itertools import chain, product
from collections import defaultdict

### Prepare data

In [3]:
# Link to NYT data folder

nyt_code_dir = "/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_code/"
FILE_NAMES = os.listdir(nyt_code_dir)

# Link to dictionary information

info_path = "/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_info/indexer_word2emb_100k.p"
indexer100k, word2emb100k = dill.load(open(info_path, 'rb'))
glove_embs = []
for i in range(len(indexer100k)):
    glove_embs.append(word2emb100k[indexer100k.get_object(i)])
glove_embs = np.array(glove_embs)
print(glove_embs.shape)

(100001, 300)


In [4]:
# Data batching

def batch_mixture(doc_a, doc_b, k=25):
    batch_x1, batch_x2, batch_y = [], [], []
    ys = [1,0,0,1]
    for _ in range(k): # 4 entries added per iteration.
        for i,(da,db) in enumerate(product([doc_a, doc_b], 
                                           [doc_a, doc_b])):
            batch_x1.append(random.choice(da))
            batch_x2.append(random.choice(db))
            batch_y.append(ys[i])
    return batch(batch_x1), batch(batch_x2), np.array(batch_y)

def get_batch(file_idx):
    filename = FILE_NAMES[file_idx]
    doc_a, doc_b, doc_mix = dill.load(open(nyt_code_dir+filename, 'rb'))
    (batch_x1,batch_x1_len), (batch_x2,batch_x2_len), batch_y = batch_mixture(doc_a,doc_b)
    return batch_x1,batch_x1_len,batch_x2,batch_x2_len,batch_y

### Bi-LSTM classifier

In [9]:
tf.reset_default_graph()

sess = tf.InteractiveSession()

VOCAB_SIZE = len(indexer100k)
EMB_SIZE = glove_embs.shape[1]
HID_SIZE = 100
NUM_LAYERS = 2
KEEP_PROB = 0.7
LEARNING_RATE = 1e-5

input_x1 = tf.placeholder(tf.int32, [None, None], name='input_x1') # <max-time, batch-size>
input_x2 = tf.placeholder(tf.int32, [None, None], name='input_x2')
input_x1_length = tf.placeholder(tf.int32, [None], name='input_x1_length')
input_x2_length = tf.placeholder(tf.int32, [None], name='input_x2_length')
input_y  = tf.placeholder(tf.int32, [None], name='input_y')

keep_prob = tf.placeholder(tf.float32, name="keep_prob")

with tf.variable_scope('Embeddings'):
    embeddings = tf.get_variable('embeddings', glove_embs.shape, 
                                 initializer=tf.contrib.layers.xavier_initializer())
    glove_init = embeddings.assign(glove_embs)
    input_x1_embedded = tf.nn.embedding_lookup(embeddings, input_x1) # <max-time, batch-size, emb-size>
    input_x2_embedded = tf.nn.embedding_lookup(embeddings, input_x2)

cell = MultiRNNCell([DropoutWrapper(LSTMCell(HID_SIZE),output_keep_prob=keep_prob)]*NUM_LAYERS) 

def run_lstm(cell, inputs, inputs_length): # lstm-out size *= 2 by bidirectionality.
    ((fw_outputs,bw_outputs), # <max-time, batch-size, hid-size>, attention later if needed.
     (fw_final_state,bw_final_state)) = ( # <batch-size, hid-size>
        tf.nn.bidirectional_dynamic_rnn(cell_fw=cell,
                                        cell_bw=cell,
                                        inputs=inputs,
                                        sequence_length=inputs_length,
                                        dtype=tf.float32, time_major=True)
    )
    return tf.concat([tf.concat([fw_state_tuple.h,bw_state_tuple.h], 1) # lstm-out size *= NUM_LAYERS by stacking.
                      for fw_state_tuple,bw_state_tuple in zip(fw_final_state,bw_final_state)], 1), \
           tf.transpose(tf.concat([fw_outputs,bw_outputs], 2), [1,0,2])
        # op1: <batch-size, hid-size*2*num-layers>
        # op2: <max-time, batch-size, hid-size*2> -> <batch-size, max-time, hid-size*2>
        # NB: batch-size = 1 here.
           
with tf.variable_scope('Bi-LSTM') as scope:
    final_state_x1, outputs_x1 = run_lstm(cell, input_x1_embedded, input_x1_length) 
        # fs_x1: <batch-size, hid-size*2*num-layers>
        # o_x1: <batch-size, max-time, hid-size*2>
    scope.reuse_variables() # both sentence inputs share the same weights.
    final_state_x2, outputs_x2 = run_lstm(cell, input_x2_embedded, input_x2_length)

def run_attention(outputs, state):
    W_d = tf.get_variable('W_d', [HID_SIZE*2, HID_SIZE*2], initializer=tf.contrib.layers.xavier_initializer())
    W_s = tf.get_variable('W_s', [HID_SIZE*2*NUM_LAYERS, HID_SIZE*2], 
                          initializer=tf.contrib.layers.xavier_initializer())
    d_W = tf.tensordot(outputs, W_d, axes=[[2],[0]])
        # <batch-size, max-time, hid-size*2> * <hid-size*2, hid-size*2> = <batch-size, max-time, hid-size*2>
    s_W = tf.expand_dims(tf.matmul(state, W_s), axis=1)
        # op1. <batch-size, hid-size*2*num-layers> * <hid-size*2*num-layers, hid-size*2> -> <batch-size, hid-size*2>
        # op2. <batch-size, hid-size*2> -> <batch-size, 1, hid-size*2>
    a_tsr = tf.nn.tanh(tf.add(d_W, s_W))
        # op1. <batch-size, max-time, hid-size*2> + <batch-size, 1, hid-size*2> -> <batch-size, max-time, hid-size*2>
        # op2. elem-wise nonlinearity.
    W_a = tf.get_variable('W_a', [HID_SIZE*2, 1], initializer=tf.contrib.layers.xavier_initializer())
    a_W = tf.nn.softmax(tf.tensordot(a_tsr, W_a, axes=[[2],[0]]), dim=1)
        # op1. <batch-size, max-time, hid-size*2> * <hid-size*2, 1> -> <batch-size, max-time, 1>
        # op2. softmax over max-time.
    d_a = tf.reduce_sum(tf.multiply(outputs, a_W), axis=1)
        # op1. <batch-size, max-time, hid-size*2> elem* <batch-size, max-time, 1> -> <batch-size, max-time, hid-size*2>
        # op2. sum over max-time (weighted sum) -> <batch-size, hid-size*2>
    return d_a    

with tf.variable_scope('Mutual-Attention') as scope:
    x1_to_x2_att = run_attention(outputs_x2, final_state_x1) # x1 attending to x2, <batch-size, hid-size*2>
    scope.reuse_variables()
    x2_to_x1_att = run_attention(outputs_x1, final_state_x2) # x2 attending to x1

def run_scores(fs_x1, fs_x2, att_12, att_21):
    W_bi = tf.get_variable('W_bi', [HID_SIZE*2*NUM_LAYERS+HID_SIZE*2, HID_SIZE*2*NUM_LAYERS+HID_SIZE*2], 
                           initializer=tf.contrib.layers.xavier_initializer())
    fv_x1 = tf.concat([fs_x1,att_12],axis=1) # <batch-size, hid-size*2>
    fv_x2 = tf.concat([fs_x2,att_21],axis=1)
    return tf.nn.sigmoid(tf.diag_part(tf.matmul(tf.matmul(fv_x1,W_bi),tf.transpose(fv_x2))),name='scores')
    
scores = run_scores(final_state_x1, final_state_x2, x1_to_x2_att, x2_to_x1_att)
predictions = tf.cast(tf.round(scores), tf.int32, name='predictions')     
    
with tf.name_scope('Loss'):
    losses = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(input_y, tf.float32), logits=scores)
    loss = tf.reduce_mean(losses, name='loss')

with tf.name_scope('Accuracy'):
    correct_predictions = tf.equal(predictions, input_y)
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')
    
global_step = tf.Variable(0, name='global_step', trainable=False)
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
grads_and_vars = optimizer.compute_gradients(loss)
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step, name='train_op')

sess.run(tf.global_variables_initializer())

# saver = tf.train.Saver()

In [1]:
NUM_EPOCHS = 1
TRAIN_SIZE = 10
# TRAIN_SIZE = len(FILE_NAMES)
VERBOSE = 1
ERROR_LOG = []

loss_track, accuracy_track = [], []
start = time.time()
try:
    for e in range(NUM_EPOCHS):
        print('Epoch ', e+1)
        print('\n')
        file_indices = np.random.choice(list(range(len(FILE_NAMES))), size=TRAIN_SIZE, replace=False)
        random.shuffle(file_indices)  
        curr_loss_track, curr_accuracy_track = [], []
        for file_idx in file_indices:
            try:
                batch_x1,batch_x1_length,batch_x2,batch_x2_length,batch_y = get_batch(file_idx)
            except:
                ERROR_LOG.append(file_idx)
                continue
            fd = {input_x1:batch_x1, input_x1_length:batch_x1_length,
                  input_x2:batch_x2, input_x2_length:batch_x2_length,
                  input_y:batch_y,
                  keep_prob:KEEP_PROB}   
            _, step, loss_, accuracy_ = sess.run([train_op, global_step, loss, accuracy], feed_dict=fd)
            curr_loss_track.append(loss_)
            curr_accuracy_track.append(accuracy_)
            if step%VERBOSE==0:
                print('  batch loss & accuracy at step {}: <{}, {}> (time elapsed = {})'.format(step, 
                                                                                                np.mean(curr_loss_track),
                                                                                                np.mean(curr_accuracy_track),
                                                                                                time.time()-start))
                start = time.time()
        print('\n')
        print('  epoch mean loss & accuracy: <{}, {}>'.format(np.mean(curr_loss_track),np.mean(curr_accuracy_track)))
        print('\n')    
        loss_track += curr_loss_track
        accuracy_track += curr_accuracy_track 
except KeyboardInterrupt:
    print('Stopped!')

### Model Saving

In [15]:
def remove_all_files(target_dir):
    for filename in os.listdir(target_dir):
        os.remove(os.path.abspath(os.path.join(target_dir, filename)))

In [16]:
save_dir = "/work/04233/sw33286/AIDA-INDIV-MODEL-SAVE/our-model-no-context-mutual-attention/"
save_path = save_dir + "our-model-no-context-mutual-attention-00"
remove_all_files(save_dir)
saver.save(sess, save_path)

'/work/04233/sw33286/AIDA-INDIV-MODEL-SAVE/our-model-no-context-mutual-attention/our-model-no-context-mutual-attention-00'

### Bi-LSTM + HAC

In [13]:
# Evaluation functions

def get_rand_mixture():
    filename = random.choice(FILE_NAMES)
    da,db, doc_mix = dill.load(open(nyt_code_dir+filename, 'rb'))
    doc_lbs = []
    for sentcode in doc_mix:
        if sentcode in da:
            doc_lbs.append(0)
        else:
            doc_lbs.append(1)
    return doc_mix, doc_lbs

def flip_clust(clust):
    return np.array([0 if i==1 else 1 for i in clust])

def clust_accuracy(true, pred):
    return max(accuracy_score(true, pred),
               accuracy_score(true, flip_clust(pred)))

def to_sent(code):
    return [indexer100k.get_object(idx) for idx in code]

# Bi-LSTM + HAC class

class ClfHAC:
    
    def __init__(self, clf_dir, clf_filename):
        self.sess = tf.Session()
        saver = tf.train.import_meta_graph(clf_dir + clf_filename)
        saver.restore(self.sess, tf.train.latest_checkpoint(clf_dir))
        self.graph = tf.get_default_graph()
        self.input_x1 = self.graph.get_tensor_by_name('input_x1:0')
        self.input_x2 = self.graph.get_tensor_by_name('input_x2:0')
        self.input_x1_length = self.graph.get_tensor_by_name('input_x1_length:0')
        self.input_x2_length = self.graph.get_tensor_by_name('input_x2_length:0')
        self.input_y = self.graph.get_tensor_by_name('input_y:0')
        self.keep_prob = self.graph.get_tensor_by_name('keep_prob:0')

        self.scores = self.graph.get_tensor_by_name('scores:0')
        self.predictions = self.graph.get_tensor_by_name('predictions:0')
        self.loss = self.graph.get_tensor_by_name('Loss/loss:0')
        self.accuracy = self.graph.get_tensor_by_name('Accuracy/accuracy:0')
        self.global_step = self.graph.get_tensor_by_name('global_step:0')
        self.train_op = self.graph.get_tensor_by_name('train_op:0')
        
        
    def dist(self, x1, x2):
        x1, x1_len = batch([x1])
        x2, x2_len = batch([x2])
        fd = {self.input_x1:x1, self.input_x1_length:x1_len,
              self.input_x2:x2, self.input_x2_length:x2_len,
              self.keep_prob:1.0}
        conf = self.sess.run(self.scores, feed_dict=fd)
        return 1-conf[0]
    
    def evaluate(self, doc_mix, doc_lbs, method='average', plot=True):
        doc_mix_sq, _ = batch(doc_mix)
        doc_mix_sq = doc_mix_sq.T
        doc_mix_clust = linkage(doc_mix_sq, method=method, metric=self.dist)
        # evaluate by class-based prec/rec/f1
        doc_prd = fcluster(doc_mix_clust, 2, criterion='maxclust') - 1 # predicted assignments (label adjusted)
        acc = clust_accuracy(doc_lbs, doc_prd)
        if plot:
            print('Clustering accuracy = {}'.format(acc))
            print('\n')
            plt.figure(figsize=(25, 10))
            plt.title('Hierarchical Clustering Dendrogram')
            plt.xlabel('sample index')
            plt.ylabel('distance')
            dendrogram(
                doc_mix_clust,
                leaf_rotation=90.,  # rotates the x axis labels
                leaf_font_size=15.,  # font size for the x axis labels
            )
            plt.show() 
            print('True | Pred | Sentence')
            for label,pred_label,code in zip(doc_lbs,doc_prd,doc_mix):
                if label==0:
                    print('\033[1;37;40m {}    | {}    | {}'.format(label,pred_label,to_sent(code)))
                else:
                    print('\033[1;30;47m {}    | {}    | {}'.format(label,pred_label,to_sent(code)))
            print('\n' + Style.RESET_ALL) 
        else:
            return doc_mix_clust, acc
        

In [5]:
restore_dir = "/work/04233/sw33286/AIDA-INDIV-MODEL-SAVE/our-model-no-context-mutual-attention/"
restore_filename = "our-model-no-context-mutual-attention-00.meta"
clf_hac = ClfHAC(restore_dir, restore_filename)

In [ ]:
clf_hac.evaluate(*get_rand_mixture())

### Evaluation on subset

In [ ]:
def rand_evaluation(k=100):
    accuracies = []
    for _ in range(k):
        _, acc = clf_hac.evaluate(*get_rand_mixture(), plot=False)
        accuracies.append(acc)
    print('Average clustering accuracy over {} samples = {}'.format(k, np.mean(accuracies)))

In [ ]:
%%time

rand_evaluation()

### *Model Load-n-Train

In [1]:
# Load facilities

import sys
sys.path.insert(0, "/work/04233/sw33286/AIDA-SCRIPTS")

import os
import time
import random
import shutil
import dill
import numpy as np

import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell, MultiRNNCell, DropoutWrapper

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline
import colorama
from colorama import Style

from helpers import Indexer, batch
from itertools import chain, product
from collections import defaultdict

# Link to NYT data folder

nyt_code_dir = "/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_code/"
FILE_NAMES = os.listdir(nyt_code_dir)

# Link to dictionary information

info_path = "/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_info/indexer_word2emb_100k.p"
indexer100k, word2emb100k = dill.load(open(info_path, 'rb'))
glove_embs = []
for i in range(len(indexer100k)):
    glove_embs.append(word2emb100k[indexer100k.get_object(i)])
glove_embs = np.array(glove_embs)
print(glove_embs.shape)

# Data batching

def batch_mixture(doc_a, doc_b, k=25):
    batch_x1, batch_x2, batch_y = [], [], []
    ys = [1,0,0,1]
    for _ in range(k): # 4 entries added per iteration.
        for i,(da,db) in enumerate(product([doc_a, doc_b], 
                                           [doc_a, doc_b])):
            batch_x1.append(random.choice(da))
            batch_x2.append(random.choice(db))
            batch_y.append(ys[i])
    return batch(batch_x1), batch(batch_x2), np.array(batch_y)

def get_batch(file_idx):
    filename = FILE_NAMES[file_idx]
    doc_a, doc_b, doc_mix = dill.load(open(nyt_code_dir+filename, 'rb'))
    (batch_x1,batch_x1_len), (batch_x2,batch_x2_len), batch_y = batch_mixture(doc_a,doc_b)
    return batch_x1,batch_x1_len,batch_x2,batch_x2_len,batch_y

(100001, 300)


In [2]:
# Load graph

restore_dir = "/work/04233/sw33286/AIDA-INDIV-MODEL-SAVE/our-model-no-context-mutual-attention/"
restore_filename = "our-model-no-context-mutual-attention-00.meta"

sess = tf.Session()
saver = tf.train.import_meta_graph(restore_dir + restore_filename)
saver.restore(sess, tf.train.latest_checkpoint(restore_dir))
graph = tf.get_default_graph()

input_x1 = graph.get_tensor_by_name('input_x1:0')
input_x2 = graph.get_tensor_by_name('input_x2:0')
input_x1_length = graph.get_tensor_by_name('input_x1_length:0')
input_x2_length = graph.get_tensor_by_name('input_x2_length:0')
input_y = graph.get_tensor_by_name('input_y:0')
keep_prob = graph.get_tensor_by_name('keep_prob:0')

scores = graph.get_tensor_by_name('scores:0')
predictions = graph.get_tensor_by_name('predictions:0')
loss = graph.get_tensor_by_name('Loss/loss:0')
accuracy = graph.get_tensor_by_name('Accuracy/accuracy:0')
global_step = graph.get_tensor_by_name('global_step:0')
train_op = graph.get_tensor_by_name('train_op:0')

In [12]:
NUM_EPOCHS = 1
TRAIN_SIZE = 10
# TRAIN_SIZE = len(FILE_NAMES)
VERBOSE = 1
ERROR_LOG = []

loss_track, accuracy_track = [], []
start = time.time()
try:
    for e in range(NUM_EPOCHS):
        print('Epoch ', e+1)
        print('\n')
        file_indices = np.random.choice(list(range(len(FILE_NAMES))), size=TRAIN_SIZE, replace=False)
        random.shuffle(file_indices)  
        curr_loss_track, curr_accuracy_track = [], []
        for file_idx in file_indices:
            try:
                batch_x1,batch_x1_length,batch_x2,batch_x2_length,batch_y = get_batch(file_idx)
            except:
                ERROR_LOG.append(file_idx)
                continue
            fd = {input_x1:batch_x1, input_x1_length:batch_x1_length,
                  input_x2:batch_x2, input_x2_length:batch_x2_length,
                  input_y:batch_y,
                  keep_prob:KEEP_PROB}   
            _, step, loss_, accuracy_ = sess.run([train_op, global_step, loss, accuracy], feed_dict=fd)
            curr_loss_track.append(loss_)
            curr_accuracy_track.append(accuracy_)
            if step%VERBOSE==0:
                print('  batch loss & accuracy at step {}: <{}, {}> (time elapsed = {})'.format(step, 
                                                                                                np.mean(curr_loss_track),
                                                                                                np.mean(curr_accuracy_track),
                                                                                                time.time()-start))
                start = time.time()
        print('\n')
        print('  epoch mean loss & accuracy: <{}, {}>'.format(np.mean(curr_loss_track),np.mean(curr_accuracy_track)))
        print('\n')    
        loss_track += curr_loss_track
        accuracy_track += curr_accuracy_track 
except KeyboardInterrupt:
    print('Stopped!')

In [ ]:
# Save

def remove_all_files(target_dir):
    for filename in os.listdir(target_dir):
        os.remove(os.path.abspath(os.path.join(target_dir, filename)))

save_dir = "/work/04233/sw33286/AIDA-INDIV-MODEL-SAVE/our-model-no-context-mutual-attention/"
save_path = save_dir + "our-model-no-context-mutual-attention-00"
remove_all_files(save_dir)
saver.save(sess, save_path)